# Karachi Weather: Interactive Time Series with Dash Python
### Project by [Razi Haider](https://www.linkedin.com/in/razi-h/)


In [2]:
import dash
import pandas as pd
from dash import dcc, html
import pandas as pd
import numpy as np
from dash.dependencies import Output, Input
from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt

In [3]:
app = dash.Dash(__name__)
app.title = "Time Series Analysis Of Karachi Weather"

data = pd.read_csv("predicted_temperatures.csv")
data["Date"] = pd.to_datetime(data["Datetime"], format="%m/%d/%Y")
data.sort_values("Date", inplace=True)
print(data.head())

app.layout = html.Div(
    children=[
        html.Div(
            children=[
                html.H1(
                    children="Time Series Analysis Of Karachi Weather", className="header-title"
                ),
                html.P(
                    children="Analyze the behavior of Karachi weather over time"
                    " with interactive visualizations"
                    " between 2021 and 2023",
                    className="header-description",
                ),
            ],
            className="header",
        ),
        html.Div(
            children=[
                html.Div(
                    children=[
                        html.Div(children="Algorithm To use", className="menu-title"),
                        dcc.Dropdown(
                            id="algorithm-filter",
                            options=[
                                {'label': 'ARIMAX', 'value': 'ARIMAX'},
                                {'label': 'SARIMAX', 'value': 'SARIMAX'},
                                {'label': 'RANDOM FOREST', 'value': 'RandomForest'},
                                {'label': 'LASSO REGRESSOR', 'value': 'LassoRegressor'},
                            ],
                            value="ARIMAX",
                            clearable=False,
                            className="dropdown",
                        ),
                    ]
                ),
                html.Div(
                    children=[
                        html.Div(
                            children="Date Range",
                            className="menu-title"
                        ),
                        dcc.DatePickerRange(
                            id="date-range",
                            min_date_allowed=data.Date.min().date(),
                            max_date_allowed=data.Date.max().date(),
                            start_date=data.Date.min().date(),
                            end_date=data.Date.max().date(),
                        ),
                    ]
                ),
            ],
            className="menu",
        ),
        html.Div(
            children=[
                html.Div(
                    children=dcc.Graph(
                        id="prediction-chart", config={"displayModeBar": False},
                    ),
                    className="card",
                ),
            ],
            className="wrapper",
        ),
    ]
)


@app.callback(
    [Output("prediction-chart", "figure")],
    [
        Input("algorithm-filter", "value"),
        Input("date-range", "start_date"),
        Input("date-range", "end_date"),
    ],
)

def update_charts(algorithm, start_date, end_date):
    mask = (
        (data.Algorithm == algorithm)
        & (data.Date >= start_date)
        & (data.Date <= end_date)
    )
    filtered_data = data.loc[mask, :]
    
    # actual_data = filtered_data.loc[filtered_data['Data_Type'] == 'Actual']
    # predicted_data = filtered_data.loc[filtered_data['Data_Type']
    #                                    == 'Predicted_Temperature']

    price_chart_figure = {
        "data": [
            {
                "x": filtered_data["Datetime"],
                "y": filtered_data["Predicted_Temperature"],
                "type": "lines",
                "hovertemplate": "%{y:.2f}C<extra></extra>",
                "name": "Predicted",
                "line": {"color": "red"}
            },
            {
                "x": filtered_data["Datetime"],
                "y": filtered_data["Actual"],
                "type": "lines",
                "hovertemplate": "%{y:.2f}C<extra></extra>",
                "name": "Actual",
                "line": {"color": "blue"}
            }
        ],
        "layout": {
            "title": {
                "text": "Predicted Temperature Over Time", "x": 0.05, "xanchor": "left",
            },
            "xaxis": {"fixedrange": True},
            "yaxis": {"fixedrange": True},
            "colorway": ["#17B897"],
        },
    }


    return [price_chart_figure]


if __name__ == "__main__":
    app.run_server(debug=True, use_reloader=False, port=8051)

     Datetime  Predicted_Temperature       Algorithm  Actual       Date
0    1/1/2023              19.142424          ARIMAX    19.0 2023-01-01
348  1/1/2023              26.519200  LassoRegressor    19.0 2023-01-01
232  1/1/2023              18.953000    RandomForest    19.0 2023-01-01
116  1/1/2023              19.323846         SARIMAX    19.0 2023-01-01
349  1/2/2023              26.519200  LassoRegressor    18.6 2023-01-02
